In [41]:
%matplotlib inline
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import pandas
import sklearn
import pickle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import learning_curve

In [43]:
! git init 
! dvc init 

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /home/jexica/Applied-ML-course/Assignment2/.git/


Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [44]:
! dvc remote add -d remote_storage gdrive://1mBSh692-OHn7R_tIz-j2QxLp6AKN91Jm
! git log
! dvc add "data/raw"
! dvc add "data/prepared"
! git add .
! git commit -m "Raw data & Initialize DVC"

Setting 'remote_storage' as a default remote.
fatal: your current branch 'master' does not have any commits yet
⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/home/jexica/Applied-ML-course/Assignment2/.dvc/cache/fi
                                                                                
!
  0%|          |Adding data/raw to cache              0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /home/jexica/Applied-ML-c0/2 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00,  3.68file/s]

To track the changes with git, run:

	git add data/raw.dvc data/.gitignore

To enable auto staging, run:

	dvc config core.autostage true
⠋ Checking graph                

In [45]:
#load the data from a given file path
data = pandas.read_csv('data/raw/emails.csv',sep=',',header=None)
data= data[1:]
data.columns=["message", "label"]
print(data)

                                                message label
1     Subject: naturally irresistible your corporate...     1
2     Subject: the stock trading gunslinger  fanny i...     1
3     Subject: unbelievable new homes made easy  im ...     1
4     Subject: 4 color printing special  request add...     1
5     Subject: do not have money , get software cds ...     1
...                                                 ...   ...
5724  Subject: re : research and development charges...     0
5725  Subject: re : receipts from visit  jim ,  than...     0
5726  Subject: re : enron case study update  wow ! a...     0
5727  Subject: re : interest  david ,  please , call...     0
5728  Subject: news : aurora 5 . 2 update  aurora ve...     0

[5728 rows x 2 columns]


In [46]:
#Preprocess the data
def split_into_lemmas(data):
    data = data.lower()
    words = TextBlob(data).words
    # for each word, take its "base form" = lemma 
    return [word.lemma for word in words]

data.message.head().apply(split_into_lemmas)
print(data)

                                                message label
1     Subject: naturally irresistible your corporate...     1
2     Subject: the stock trading gunslinger  fanny i...     1
3     Subject: unbelievable new homes made easy  im ...     1
4     Subject: 4 color printing special  request add...     1
5     Subject: do not have money , get software cds ...     1
...                                                 ...   ...
5724  Subject: re : research and development charges...     0
5725  Subject: re : receipts from visit  jim ,  than...     0
5726  Subject: re : enron case study update  wow ! a...     0
5727  Subject: re : interest  david ,  please , call...     0
5728  Subject: news : aurora 5 . 2 update  aurora ve...     0

[5728 rows x 2 columns]


In [47]:
#First split the data into train/validation/test
train_val_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, validation_data = train_test_split(train_val_data, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2
train_data.to_csv('data/prepared/train.csv', index=False)
validation_data.to_csv('data/prepared/validation.csv', index=False)
test_data.to_csv('data/prepared/test.csv', index=False)

In [48]:
! dvc status
! dvc add "data/prepared"
! git add .
! dvc push
! git commit -m "Adding the first version of split data"

data/prepared.dvc:                                                              
	changed outs:
		modified:           data/prepared
⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/home/jexica/Applied-ML-course/Assignment2/.dvc/cache/fi
                                                                                
!
  0%|          |Adding data/prepared to cache         0/3 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /home/jexica/Applied-ML-c0/4 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 10.84file/s]

To track the changes with git, run:

	git add data/prepared.dvc

To enable auto staging, run:

	dvc config core.autostage true
Pushing
!
  0% Querying

# Update Version with different seed 


In [49]:
#Updated split of the data into train/validation/test
train_val_data, test_data = train_test_split(data, test_size=0.2, random_state=73)
train_data, validation_data = train_test_split(train_val_data, test_size=0.25, random_state=73)  # 0.25 x 0.8 = 0.2
train_data.to_csv('data/prepared/train.csv', index=False)
validation_data.to_csv('data/prepared/validation.csv', index=False)
test_data.to_csv('data/prepared/test.csv', index=False)

In [50]:
! dvc status
! dvc add "data/prepared"
! git add .
! dvc push
! git commit -m "Updated version of split data using different seed"

data/prepared.dvc:                                                              
	changed outs:
		modified:           data/prepared
⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/home/jexica/Applied-ML-course/Assignment2/.dvc/cache/fi
                                                                                
!
  0%|          |Adding data/prepared to cache         0/3 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /home/jexica/Applied-ML-c0/4 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 11.70file/s]

To track the changes with git, run:

	git add data/prepared.dvc

To enable auto staging, run:

	dvc config core.autostage true
Pushing
!
  0% Querying

# Target Variable Distribution

In [51]:
def print_label_distribution(data, filename):
    print("Number of 0s in " + filename + " - " + str((data['label']== 0).sum()))
    print("Number of 1s in " + filename + " - " + str((data['label']== 1).sum()))

First Version

In [52]:
! git log --oneline

87924f3 (HEAD -> master) Updated version of split data using different seed
fe7c6b6 Adding the first version of split data
20ac06b Raw data & Initialize DVC


In [54]:
! git checkout fe7c6b6 "data/prepared.dvc"
! dvc checkout

Updated 1 path from fc34982


Building workspace index                              |9.00 [00:00,  506entry/s]
Comparing indexes                                     |8.00 [00:00,  155entry/s]
Applying changes                                      |3.00 [00:00,  81.8file/s]
M       data/prepared/


In [55]:
train_data = pandas.read_csv('data/prepared/train.csv')
validation_data = pandas.read_csv('data/prepared/validation.csv')
test_data = pandas.read_csv('data/prepared/test.csv')
print_label_distribution(train_data, "train.csv")
print_label_distribution(validation_data, "validation.csv")
print_label_distribution(test_data, "test.csv")

Number of 0s in train.csv - 2619
Number of 1s in train.csv - 817
Number of 0s in validation.csv - 885
Number of 1s in validation.csv - 261
Number of 0s in test.csv - 856
Number of 1s in test.csv - 290


In [56]:
! git log --oneline

87924f3 (HEAD -> master) Updated version of split data using different seed
fe7c6b6 Adding the first version of split data
20ac06b Raw data & Initialize DVC


In [57]:
! git checkout 87924f3 "data/prepared.dvc"
! dvc checkout

Updated 1 path from 64ff5aa


Building workspace index                              |9.00 [00:00,  625entry/s]
Comparing indexes                                    |8.00 [00:00, 1.10kentry/s]
Applying changes                                      |3.00 [00:00,  59.5file/s]
M       data/prepared/


In [58]:
train_data = pandas.read_csv('data/prepared/train.csv')
validation_data = pandas.read_csv('data/prepared/validation.csv')
test_data = pandas.read_csv('data/prepared/test.csv')
print_label_distribution(train_data, "train.csv")
print_label_distribution(validation_data, "validation.csv")
print_label_distribution(test_data, "test.csv")

Number of 0s in train.csv - 2624
Number of 1s in train.csv - 812
Number of 0s in validation.csv - 870
Number of 1s in validation.csv - 276
Number of 0s in test.csv - 866
Number of 1s in test.csv - 280
